# Spatial Correspondance Using Ray Casting of Ischemic and Territory Regions on the Surface of the Myocardium

Import the files and change the parameters based on the scale of the image.

In [223]:
import os
import vtk
import argparse
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from utilities import ReadVTUFile, ReadVTPFile, WriteVTPFile, GetCentroid, ThresholdPointsByUpper, LargestConnectedRegion, PrintProgress


InputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/RayBasedIschemicProfile/SU26A/LAD"
InputMBFBase = f"{InputFolder}/UpperMyocardium.vtu"
InputMBFApex = f"{InputFolder}/Apex.vtu"
InputIschemic = f"{InputFolder}/Ischemic.vtu"

NRaySection = 1500
NRaySphere = 40000
NSection = 100
TerritoryTag = ['LAD_0', 'Diag']

scale = 'cm'

if scale == 'cm':
    Radius = 3.5
    L_Section_Ray = 8
    L_Sphere_Ray = 5
    Ray_Thickness = 0.02

elif scale == 'mm':
    Radius = 35
    L_Section_Ray = 100
    L_Sphere_Ray = 50
    Ray_Thickness = 0.2

else:
    print("provide a proper scale")



### Defining necessary functions

In [224]:

def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    
    return line.GetOutput()

def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(Ray_Thickness)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOn()  # Close tube ends
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def SliceWPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    
    return Slice.GetOutput()

def ClipWPlane(surface, center, axis):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(surface)
    clipper.SetClipFunction(plane)
    clipper.GenerateClippedOutputOff()
    clipper.InsideOutOff()
    #clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def RotateVector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray*np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    
    return ray_rot

def ProbeFilter(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    
    return probe.GetOutput()

def IncreaseResolution(surface, subdivisions = 3):
    subdivide = vtk.vtkLoopSubdivisionFilter()
    subdivide.SetInputData(surface)
    subdivide.SetNumberOfSubdivisions(subdivisions)
    subdivide.Update()

    return subdivide.GetOutput()

def Cylinder(CenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(CenterLine)
    tube_filter.SetRadius(Radius)
    tube_filter.SetNumberOfSides(2*NRaySection)
    tube_filter.CappingOff()
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def Hemisphere(apex_centeroid, CL_axis):
    low_res = NRaySphere // 10
    
    sphere = vtk.vtkSphereSource()
    sphere.SetCenter(apex_centeroid)
    sphere.SetRadius(Radius)
    sphere.SetPhiResolution(low_res)
    sphere.SetThetaResolution(low_res)
    sphere.Update()

    Hemisphere = ClipWPlane(sphere.GetOutput(), apex_centeroid, CL_axis)
    

    return Hemisphere

def fibonacci_sphere(samples=100):
    """Generates evenly distributed unit vectors over a sphere"""
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # Golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = np.sqrt(1 - y * y)  # Radius at given y
        theta = phi * i  # Golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius
        points.append([x, y, z])

    return np.array(points)


In [225]:
slice_apex = f"{InputFolder}/Slice_Apex.vtp"
slice_base = f"{InputFolder}/Slice_Base.vtp"

SliceApex = ReadVTPFile(slice_apex)
SliceBase = ReadVTPFile(slice_base)

base_centeroid = GetCentroid(SliceBase)
apex_centeroid = GetCentroid(SliceApex)

AnnotationPoints = [base_centeroid, apex_centeroid]

centerline_axis = np.array([AnnotationPoints[1][0] - AnnotationPoints[0][0], 
                                AnnotationPoints[1][1] - AnnotationPoints[0][1], 
                                AnnotationPoints[1][2] - AnnotationPoints[0][2]])

CL_axis = centerline_axis/np.linalg.norm(centerline_axis)

MBF = ReadVTUFile(InputMBFBase)
Apex = ReadVTUFile(InputMBFApex)

"""
Myocardium_centeroid = GetCentroid(self.MBF)
point1 = np.array([Myocardium_centeroid[0] + self.CL_axis[0]*60, Myocardium_centeroid[1] + self.CL_axis[1]*60, Myocardium_centeroid[2] + self.CL_axis[2]*60])
point2 = np.array([Myocardium_centeroid[0] - self.CL_axis[0]*50, Myocardium_centeroid[1] - self.CL_axis[1]*50, Myocardium_centeroid[2] - self.CL_axis[2]*50])
"""

Ischemic = LargestConnectedRegion(ReadVTUFile(InputIschemic))

CenterLine = Line(AnnotationPoints[0], AnnotationPoints[1], NSection)

In [226]:
def CastRaysVisualizations(CLPoints):
    
    #>>> Ray Casting across the Myocardium Cylinder 
    slices = vtk.vtkAppendPolyData()

    NRays_sample = 20
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays_sample, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    for k in range(0,len(CLPoints), int(len(CLPoints)/(NSection/10))):
        Myo_slice = SliceWPlane(MBF, CLPoints[k], CL_axis)
        slices.AddInputData(Myo_slice)

        origin = CLPoints[k]
        slice_rays = vtk.vtkAppendPolyData()
        for i in range(NRays_sample):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, Ischemic)
            
            ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            is_ischemic = ischemic_profile[ischemic_profile > 0]
            if is_ischemic.size > 0:
                ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

            ray_projected = ProbeFilter(ray_projected, MBF)
            
            ischemic_profile.SetName("IschemicProfile")
            ray_projected.GetPointData().AddArray(ischemic_profile)

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")
            ray_projected.GetPointData().AddArray(territory_profile)
            slice_rays.AddInputData(BoldLine(ray_projected))
    
        slice_rays.Update()
    
        Rays.AddInputData(slice_rays.GetOutput())


    #>>> Ray Casting across the Apex Hemisphere
    NRays_sample = 100
    directions = fibonacci_sphere(NRays_sample)
    origin = apex_centeroid
    Rays_ = vtk.vtkAppendPolyData()

    for i in range(NRays_sample):
        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, Ischemic)
        
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
        is_ischemic = ischemic_profile[ischemic_profile > 0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

        ray_projected = ProbeFilter(ray_projected, Apex)
        
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)
        Rays_.AddInputData(BoldLine(ray_projected))


    Rays_.Update()
    Rays_ = ClipWPlane(Rays_.GetOutput(), apex_centeroid, CL_axis)
    Rays.AddInputData(Rays_)
    Rays.Update()
    ray_path = f"{InputFolder}/Rays.vtp"
    WriteVTPFile(ray_path, Rays.GetOutput())


    slices.Update()
    slice_path = f"{InputFolder}/MyoSlices.vtp"
    WriteVTPFile(slice_path, slices.GetOutput())


In [240]:
def RayCastingAlongMyocardium(CLPoints):

    NRays = NRaySection
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    print("------ Ray Casting across Sections along Myocardium:")
    progress_old = -1
    for k in range(len(CLPoints)):
        progress = PrintProgress(k,len(CLPoints),progress_old)
        progress_old = progress

        origin = CLPoints[k]
        slice_rays = vtk.vtkAppendPolyData()
        
        for i in range(NRays):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, Ischemic)
            
            ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            is_ischemic = ischemic_profile[ischemic_profile > 0]
            if is_ischemic.size > 0:
                ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

            ray_projected = ProbeFilter(ray_projected, MBF)
            
            ischemic_profile.SetName("IschemicProfile")
            ray_projected.GetPointData().AddArray(ischemic_profile)

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > -1]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            #else:
            #    territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")
            ray_projected.GetPointData().AddArray(territory_profile)
            slice_rays.AddInputData(ray_projected)
        slice_rays.Update()
    
        Rays.AddInputData(slice_rays.GetOutput())
    Rays.Update()

    return Rays.GetOutput()

In [241]:
def RayCastingAcrossSphere():
    directions = fibonacci_sphere(NRaySphere)
    origin = apex_centeroid
    Rays_Hemisphere = vtk.vtkAppendPolyData()
    res = 50

    print("------ Ray Casting across the Apex Hemisphere:")
    progress_old = -1
    for i in range(NRaySphere):
        progress = PrintProgress(i, NRaySphere, progress_old)
        progress_old = progress

        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, Ischemic)
        
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
        is_ischemic = ischemic_profile[ischemic_profile > 0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

        ray_projected = ProbeFilter(ray_projected, Apex)
        
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > -1]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        #else:
        #    territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)
        Rays_Hemisphere.AddInputData(ray_projected)


    Rays_Hemisphere.Update()
    #print("------ Clipping Sphere")
    #Rays_ = self.ClipWPlane(Rays_.GetOutput(), self.apex_centeroid, self.CL_axis)
    

    return Rays_Hemisphere.GetOutput()


In [229]:

def SpatialCorrespondance(Cylinder_Projected):
    with open(f"{InputFolder}/MBF_Territories_Labels.dat",'r') as infile:
        infile.readline()
        TerritoryLabels=[]
        TerritoryNames = ""
        for LINE in infile:
            line=LINE.split()
            for tag in TerritoryTag:
                if line[1].find(tag)>=0: 
                    TerritoryLabels.append(int(line[0]))
                    TerritoryNames += tag + "_"
    print(TerritoryNames)

    ThresholdArray = np.zeros(Cylinder_Projected.GetNumberOfPoints())
    for i in range(Cylinder_Projected.GetNumberOfPoints()):
        if int(Cylinder_Projected.GetPointData().GetArray("TerritoryProfile").GetValue(i)) in TerritoryLabels:
            ThresholdArray[i] = 1
    
    ThresholdArrayVTK = numpy_to_vtk(ThresholdArray, deep=True)
    ThresholdArrayVTK.SetName(f"TerritoryLabels_{TerritoryNames}")
    Cylinder_Projected.GetPointData().AddArray(ThresholdArrayVTK)
    Cylinder_Projected.Modified()

    TerritoryRegions = ThresholdPointsByUpper(Cylinder_Projected, f"TerritoryLabels_{TerritoryNames}", 1)
    WriteVTPFile(InputFolder + f"/Projected_Territories_{TerritoryNames}.vtp", TerritoryRegions)

    IschemicRegions = ThresholdPointsByUpper(Cylinder_Projected, "IschemicProfile", 1)
    ischemic_surface_path = os.path.splitext(os.path.basename(InputIschemic))[0]
    WriteVTPFile(InputFolder + f"/{ischemic_surface_path}.vtp", IschemicRegions)

    TerritoryRegionsPoints=vtk_to_numpy(TerritoryRegions.GetPoints().GetData())
    IschemicRegionsPoints=vtk_to_numpy(IschemicRegions.GetPoints().GetData())


    TerritoryTesselationString=[]
    TerritoryIschemicString=[]
    for i in range(TerritoryRegions.GetNumberOfPoints()): TerritoryTesselationString.append(str(TerritoryRegionsPoints[i]))
    for i in range(IschemicRegions.GetNumberOfPoints()): TerritoryIschemicString.append(str(IschemicRegionsPoints[i]))
    TotalPointsString=TerritoryTesselationString+TerritoryIschemicString
    UniquePointsString=set(TotalPointsString)
    OverlapCounter=len(TotalPointsString)-len(UniquePointsString)

    DiceScore = (2*OverlapCounter)/(TerritoryRegions.GetNumberOfPoints() + IschemicRegions.GetNumberOfPoints())
    print(DiceScore)
    return Cylinder_Projected


### Implementing the Method

In [230]:
print("--- Extracting the Centerline of the Myocardium")
cl_file = f"{InputFolder}/CenterLine.vtp"
WriteVTPFile(cl_file, BoldLine(CenterLine))

print("--- Visualizing Ray Casting")
CLPoints = CenterLine.GetPoints()
CLPointsArray = np.array([CLPoints.GetPoint(i) for i in range(CLPoints.GetNumberOfPoints())])
CastRaysVisualizations(CLPointsArray)


--- Extracting the Centerline of the Myocardium
--- Visualizing Ray Casting


In [242]:
print("--- Ray Casting Across the Myocardium")
Rays_Myocardium = RayCastingAlongMyocardium(CLPointsArray)


--- Ray Casting Across the Myocardium
------ Ray Casting across Sections along Myocardium:
    Progress: 0%
    Progress: 10%
    Progress: 20%
    Progress: 30%
    Progress: 40%
    Progress: 50%
    Progress: 60%
    Progress: 70%
    Progress: 80%
    Progress: 90%


In [243]:
WriteVTPFile(f"{InputFolder}/Rays_Myocardium.vtp", Rays_Myocardium)

In [244]:
Rays_Sphere = RayCastingAcrossSphere()
#print(Rays_Sphere)

------ Ray Casting across the Apex Hemisphere:
    Progress: 0%
    Progress: 10%
    Progress: 20%
    Progress: 30%
    Progress: 40%
    Progress: 50%
    Progress: 60%
    Progress: 70%
    Progress: 80%
    Progress: 90%
    Progress: 100%


In [245]:
WriteVTPFile(f"{InputFolder}/Rays_Sphere.vtp", Rays_Sphere)

### Skip running this part and Move on to the next

In [ ]:
Cylinder_Output = Cylinder(CenterLine)
Hemisphere_Output = Hemisphere(apex_centeroid, CL_axis)

WriteVTPFile(f"{InputFolder}/Cylinder_raw.vtp", Cylinder_Output)
WriteVTPFile(f"{InputFolder}/Hemisphere_raw.vtp", Hemisphere_Output)

In [ ]:
print(Hemisphere_Output.GetNumberOfPoints())

In [ ]:
Cylinder_Projected = ProbeFilter(Cylinder_Output, Rays_Myocardium)

In [ ]:
Hemisphere_Projected = ProbeFilter(Hemisphere_Output, Rays_Sphere)

In [ ]:
print("--- Writing the Output Surface")
OutputSurface = vtk.vtkAppendPolyData()
OutputSurface.AddInputData(Cylinder_Projected)
OutputSurface.AddInputData(Hemisphere_Projected)
OutputSurface.Update()

In [ ]:
print("--- Calculating the Spatial Correspondeance")
OSurface_Projected = SpatialCorrespondance(OutputSurface.GetOutput())

output_path = f"{InputFolder}/Output_Projected.vtp"
WriteVTPFile(output_path, OSurface_Projected)

In [ ]:
Rays_Hemisphere = ClipWPlane(Rays_Sphere, apex_centeroid, CL_axis)

### Here comes the next part
Manually clip Rays_Sphere to make the Rays_Hemisphere and load it to calculate the dice score.

In [248]:
Rays_Hemisphere = ReadVTPFile(f"{InputFolder}/Rays_Hemisphere.vtp")
Rays_Myocardium = ReadVTPFile(F"{InputFolder}/Rays_Myocardium.vtp")


In [ ]:
print("--- Writing the Output Surface")
OutputSurface = vtk.vtkAppendPolyData()
OutputSurface.AddInputData(Rays_Myocardium)
OutputSurface.AddInputData(Rays_Hemisphere)
OutputSurface.Update()


OSurface_Projected = SpatialCorrespondance(OutputSurface.GetOutput())

--- Writing the Output Surface
LAD_0_Diag_
